In [80]:
import time
import numpy as np
import pandas as pd
# import seaborn as sns
import string
import re
# sns.set()

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings("ignore")

# Graphics in retina format are more sharp and legible
%config InlineBackend.figure_format = 'retina'

In [81]:
df = pd.read_json('data/val.model-aware.v2.json')
df["label"] = df["label"].map({"Hallucination":1, "Not Hallucination":0})

df = df.dropna(how="any", axis=1)
df.head()

print(df['label'].value_counts())
# training using original, 0 = 295, 1 = 206

label
0    295
1    206
Name: count, dtype: int64


In [82]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
# stemmer = nltk.SnowballStemmer("english")



def clean_text(text):
    '''
    Make text lowercase, remove text in square brackets, remove links, remove punctuation,
    and remove words containing numbers.
    '''
    text = str(text).lower()
    
    # Remove text in square brackets
    text = re.sub(r'\[.*?\]', '', text)
    
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove punctuation
    text = re.sub(rf'[{re.escape(string.punctuation)}]', ' ', text)
    
    # Remove new lines
    text = re.sub(r'\n', ' ', text)
    
    # Remove words containing numbers
    text = re.sub(r'\w*\d\w*', '', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def preprocess_data(text):
    text = clean_text(text)  # Clean punctuation, URLs, and so on
    
    # Remove stopwords
    text = ' '.join(word for word in text.split() if word not in stop_words)
    
    # Lemmatize all the words in the sentence
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())
    
    return text

# Example usage
example_text = "This is an example text with a URL: http://example.com and an email: example@example.com."
cleaned_text = preprocess_data(example_text)
print(cleaned_text)


example text url email example example com


In [83]:
columns = ['hyp','src','tgt'] # exclude task
for x in columns:
    df[x] = df[x].apply(preprocess_data)

In [84]:
df = df.drop(['labels'], axis=1)
dm_df = df[df['task'] == 'DM']
mt_df = df[df['task'] == 'MT']
pg_df = df[df['task'] == 'PG']

In [141]:
# data with scores with either a 0 or 1
dm_df_highscore = dm_df[dm_df['p(Hallucination)'].isin([0.0, 1.0])]
pg_df_highscore = pg_df[pg_df['p(Hallucination)'].isin([0.0, 1.0])]
mt_df_highscore = mt_df[mt_df['p(Hallucination)'].isin([0.0, 1.0])]
dm_df_highscore['p(Hallucination)'].value_counts()
# dm_df_highscore, 1.0 = 35, 0.0 = 31
# pg_df_highscore['p(Hallucination)'].value_counts()
# pg_df_highscore, 0.0 = 43, 1.0 = 5
# mt_df_highscore['p(Hallucination)'].value_counts()
# mt_df_highscore, 0.0 = 29, 1.0 = 24

p(Hallucination)
1.0    35
0.0    31
Name: count, dtype: int64

In [136]:
pg_df_highscore

,hyp,ref,src,tgt,model,task,label,p(Hallucination)
189,debate friday vote later date kept per agenda,src,propose therefore keep arrangement per agenda ...,,tuner007/pegasus_paraphrase,PG,0,0.0
190,iraq become prominent international agenda pas...,src,past week iraq figured prominently minister sa...,,tuner007/pegasus_paraphrase,PG,0,0.0
192,achievement enlargement historic,src,enlargement accomplishment historic significance,,tuner007/pegasus_paraphrase,PG,0,0.0
193,eurobonds example second one,src,second example relates eurobonds,,tuner007/pegasus_paraphrase,PG,0,0.0
194,mr doyle want ask supplementary question,src,wish pose supplementary question mr doyle,,tuner007/pegasus_paraphrase,PG,0,0.0
195,clear europe inclusive,src,parity clear signal inclusive europe,,tuner007/pegasus_paraphrase,PG,0,0.0
196,want tell mr colom know win,src,mr president like tell mr colom naval indeed k...,,tuner007/pegasus_paraphrase,PG,1,1.0
197,clear case,src,already clear,,tuner007/pegasus_paraphrase,PG,0,0.0
201,question surrounding eurostat raised two year,src,would draw attention fact question surrounding...,,tuner007/pegasus_paraphrase,PG,0,0.0
202,fact show make worse,src,exacerbate fact situation showing,,tuner007/pegasus_paraphrase,PG,0,0.0


In [133]:
# save before_dataaug files based on task, that only has scores 0.0 or 1.0
# dm_df_highscore.to_json("dm_df_highscore.json")
# pg_df_highscore.to_json("pg_df_highscore.json")
# mt_df_highscore.to_json("mt_df_highscore.json")

In [85]:
# change name of dataset here, continue using same name balanced_dataset
# balanced_dataset = dm_df
# balanced_dataset = pg_df
balanced_dataset = mt_df

In [86]:
# create agg column with tgt + hyp
balanced_dataset["agg"] = "hyp: " + balanced_dataset["hyp"] + \
    " tgt: " + balanced_dataset["tgt"]

In [87]:
# original dataset before data aug
# X, y = pg_df['hyp'], pg_df['label']
# X, y = dm_df['hyp'], dm_df['label']
# X, y = mt_df['hyp'], mt_df['label']

# task PG
# X, y = pg_rd[['hyp','tgt']] , pg_rd['label']
# X, y = pg_ri[['hyp','tgt']] , pg_ri['label']
# X, y = pg_rs[['hyp','tgt']] , pg_rs['label']
# X, y = pg_sr[['hyp','tgt']] , pg_sr['label']
# task DM
# X, y = dm_rd[['hyp','tgt']] , dm_rd['label']
# X, y = dm_ri[['hyp','tgt']], dm_ri['label']
# X, y = dm_rs[['hyp','tgt']] , dm_rs['label']
# X, y = dm_sr[['hyp','tgt']] , dm_sr['label']
# task MT
# X, y = mt_rd[['hyp','tgt']] , mt_rd['label']
# X, y = mt_ri[['hyp','tgt']] , mt_ri['label']
# X, y = mt_rs[['hyp','tgt']] , mt_rs['label']
# X, y = mt_sr[['hyp','tgt']] , mt_sr['label']

# X, y = df['hyp'], df['label']
# one way of getting label where task is DM, df.loc[df['task'] == 'DM', 'label']
# X, y = df[df['task'] == 'PG']['hyp'], df[df['task'] == 'PG']['label']

# task PG
# X, y = pg_rd_160[['hyp','tgt']] , pg_rd_160['label']
# X, y = pg_ri_160[['hyp','tgt']] , pg_ri_160['label']
# X, y = pg_rs_160[['hyp','tgt']] , pg_rs_160['label']
# X, y = pg_sr_160[['hyp','tgt']] , pg_sr_160['label']
# pg_sr_160 for some reason does not have any tgt values in all rows??

# task DM
# X, y = dm_rd_160[['hyp','tgt']] , dm_rd_160['label']
# X, y = dm_ri_160[['hyp','tgt']] , dm_ri_160['label']
# X, y = dm_rs_160[['hyp','tgt']] , dm_rs_160['label']
# X, y = dm_sr_160[['hyp','tgt']] , dm_sr_160['label']
X, y = balanced_dataset['agg'] , balanced_dataset['label']
# dm_sr_160 had basically the same amount of entrie of class 0 and 1
# Counter({0: 4158, 1: 3738}), so i skipped making balanced dataset
# task MT
# X, y = mt_rd_160[['hyp','tgt']] , mt_rd_160['label']
# X, y = mt_ri_160[['hyp','tgt']] , mt_ri_160['label']
# X, y = mt_rs_160[['hyp','tgt']] , mt_rs_160['label']
# X, y = mt_sr_160[['hyp','tgt']] , mt_sr_160['label']

In [88]:
import collections
# y.value_counts()
counter = collections.Counter(y)
counter
# just as reference, pg_rd from thesis_data_after directory has 750 entry in total
# 480 is 0 and 270 is 1
# but pg_rs after lemmatizing and using 160 data aug,
# total: about 5200, 0 is 4158 and 1is 1092

Counter({0: 97, 1: 91})

In [89]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
# stemmer = nltk.SnowballStemmer("english")



def clean_text(text):
    '''
    Make text lowercase, remove text in square brackets, remove links, remove punctuation,
    and remove words containing numbers.
    '''
    text = str(text).lower()
    
    # Remove text in square brackets
    text = re.sub(r'\[.*?\]', '', text)
    
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove punctuation
    text = re.sub(rf'[{re.escape(string.punctuation)}]', ' ', text)
    
    # Remove new lines
    text = re.sub(r'\n', ' ', text)
    
    # Remove words containing numbers
    text = re.sub(r'\w*\d\w*', '', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def preprocess_data(text):
    text = clean_text(text)  # Clean punctuation, URLs, and so on
    
    # Remove stopwords
    text = ' '.join(word for word in text.split() if word not in stop_words)
    
    # Lemmatize all the words in the sentence
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())
    
    return text

# Example usage
example_text = "This is an example text with a URL: http://example.com and an email: example@example.com."
cleaned_text = preprocess_data(example_text)
print(cleaned_text)


example text url email example example com


In [90]:
y.shape

(188,)

In [91]:
type(X)

pandas.core.series.Series

In [92]:
# split into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,  random_state=42)

In [93]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

# paths to final test data
final_ag_path = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/final_test_data/test.model-agnostic.json'
final_aw_path = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/final_test_data/test.model-agnostic.json'

final_ag_df = pd.read_json(final_ag_path, encoding_errors='backslashreplace')
final_aw_df = pd.read_json(final_aw_path, encoding_errors='backslashreplace')
final_ag_df["label"] = final_ag_df["label"].map({"Hallucination":1, "Not Hallucination":0})
final_aw_df["label"] = final_aw_df["label"].map({"Hallucination":1, "Not Hallucination":0})
# perform preprocessing on final_df AND split it based on tasks!
columns = ['hyp','src','tgt'] # exclude task
for x in columns:
    final_ag_df[x] = final_ag_df[x].apply(preprocess_data)
for x in columns:
    final_aw_df[x] = final_aw_df[x].apply(preprocess_data)
final_ag_pg = final_ag_df[final_ag_df['task'] == 'PG']
final_ag_dm = final_ag_df[final_ag_df['task'] == 'DM']
final_ag_mt = final_ag_df[final_ag_df['task'] == 'MT']

final_aw_pg = final_aw_df[final_aw_df['task'] == 'PG']
final_aw_dm = final_aw_df[final_aw_df['task'] == 'DM']
final_aw_mt = final_aw_df[final_aw_df['task'] == 'MT']


In [94]:
from transformers import AutoTokenizer, TFRobertaForSequenceClassification

# Load tokenizer and model
checkpoint = 'FacebookAI/roberta-base'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFRobertaForSequenceClassification.from_pretrained(checkpoint, num_labels=1)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [95]:
import datasets
dataset = datasets.Dataset.from_pandas(balanced_dataset)
BATCH_SIZE=8
# Define the tokenization function
def tokenize_function(example):
    return tokenizer(example['agg'], truncation=True, padding='max_length', max_length=256)

# Tokenize the dataset
# tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_datasets_mapped = dataset.map(tokenize_function, batched=True, batch_size=BATCH_SIZE, remove_columns=['hyp','tgt'])

Map: 100%|██████████| 188/188 [00:00<00:00, 2421.81 examples/s]


In [96]:
len(dataset)

188

In [97]:
from sklearn.manifold import TSNE
import tensorflow as tf

# Extract embeddings function
def extract_embeddings(inputs):
    input_ids = tf.convert_to_tensor(inputs['input_ids'], dtype=tf.int32)
    attention_mask = tf.convert_to_tensor(inputs['attention_mask'], dtype=tf.int32)
    outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
    hidden_states = outputs.hidden_states[-1]
    return hidden_states[:, 0, :].numpy()

# i was thinking of splitting embeddings or dataset to train and test but...maybe laters
embeddings = extract_embeddings(tokenized_datasets_mapped)
# embeddings2 = extract_embeddings(tokenized_datasets_mapped)
# embeddings3 = extract_embeddings(tokenized_datasets_mapped)



In [98]:
len(embeddings[0])

768

In [99]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
import numpy as np

# Hyperparameter Tuning with Grid Search
param_grid = {
    'max_depth': [3, 4, 5],
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2]
}
xgb = XGBClassifier()
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(embeddings, y)



GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 4, 5],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [108]:
# change this place to change final_dataset
# copy_final_dataset = final_ag_pg
# copy_final_dataset = final_ag_dm
copy_final_dataset = final_ag_mt
copy_final_dataset["agg"] = "hyp: " + copy_final_dataset["hyp"] + \
    " tgt: " + copy_final_dataset["tgt"] + \
    " src: " + copy_final_dataset["src"]
X_final, y_final = copy_final_dataset['agg'], copy_final_dataset['label']
final_dataset = datasets.Dataset.from_pandas(copy_final_dataset)


final_tokenized = final_dataset.map(tokenize_function, batched=True, batch_size=BATCH_SIZE, remove_columns=['hyp','tgt'])
final_embeddings = extract_embeddings(final_tokenized)

Map: 100%|██████████| 562/562 [00:00<00:00, 2442.50 examples/s]


In [109]:
labels = collections.Counter(final_ag_pg['label'])
labels

Counter({0: 278, 1: 97})

In [110]:
best_xgb = grid_search.best_estimator_
y_pred_best = best_xgb.predict(final_embeddings)
print("Best Accuracy:", accuracy_score(y_final, y_pred_best))
print("Best Confusion Matrix:\n", confusion_matrix(y_final, y_pred_best))
print("Best Classification Report:\n", classification_report(y_final, y_pred_best))
# all 500,500 entries
# dm on pg = 0.64
# dm on dm = 0.560
# dm on mt = 0.559

# pg on pg = 0.584
# pg on dm = 0.476
# pg on mt = 0.407

# mt on pg = 0.730
# mt on dm = 0.524
# mt on mt = 0.562

# not using data augmentation

# dm on pg = 0.656 CM = 212, 66, 63, 34
# dm on dm = 0.556 CM = 86, 189, 61, 227
# dm on mt = 0.470 CM = 102, 234, 64, 162

# for CM, the first two numbers are prediction for 0, and the latter two for 1
# for example final_ag_pg has 0: 278, 1:97 and pg on pg here just predicted all 370 entries as 0
# pg on pg = 0.741 CM = 278, 0,  97, 0
# pg on dm = 0.488 CM = 275, 0, 288, 0
# pg on mt = 0.598 CM = 336, 0, 226, 0

# mt on pg = 0.731, CM = 263, 15, 86, 11
# mt on dm = 0.522, CM = 58, 217, 52, 236
# mt on mt = 0.493, CM = 308, 28, 201, 25

# using only data with scores 0 or 1!, from data augmentation!
# which would warrant the need to count amount of data with only 0 and 1 before and after data aug


Best Accuracy: 0.5925266903914591
Best Confusion Matrix:
 [[308  28]
 [201  25]]
Best Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.92      0.73       336
           1       0.47      0.11      0.18       226

    accuracy                           0.59       562
   macro avg       0.54      0.51      0.45       562
weighted avg       0.55      0.59      0.51       562



In [103]:
from collections import Counter
final_binary_counter = Counter(y_pred_best)
final_binary_counter

Counter({0: 349, 1: 26})

In [104]:
y_pred_best = best_xgb.predict(final_embeddings)
print("Best Accuracy:", accuracy_score(y_final, y_pred_best))
print("Best Confusion Matrix:\n", confusion_matrix(y_final, y_pred_best))
print("Best Classification Report:\n", classification_report(y_final, y_pred_best))

Best Accuracy: 0.7306666666666667
Best Confusion Matrix:
 [[263  15]
 [ 86  11]]
Best Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.95      0.84       278
           1       0.42      0.11      0.18        97

    accuracy                           0.73       375
   macro avg       0.59      0.53      0.51       375
weighted avg       0.67      0.73      0.67       375

